# Retrieves song features from the Spotify API

In [51]:
from __future__ import print_function    # (at top of module)
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import numpy as np

In [75]:
client_credentials_manager = SpotifyClientCredentials(client_id='c979f37a064e455e8778dbcd331c2d5d', 
                                                      client_secret='dc6fcb571e72436e8d13df8e6799b833')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [76]:
df = pd.read_csv("top_200_daily.csv")
df = df.iloc[0:200]
df["Position"] = pd.to_numeric(df["Position"])

In [77]:
features = []

i = 0
while i < len(df): 
    features.append(sp.audio_features(df["spotify_id"][i]))
    i += 1
    
features = [item for items in features for item in items] #flattens list
audio_features = pd.DataFrame(features)
audio_features.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1, inplace=True)

songs = audio_features.join(df["Position"])
songs = songs.join(df["Artist"])
songs = songs.join(df["Track Name"])

retrying ...7secs


In [83]:
chunks = []
ranking = 5

for g, df in songs.groupby(np.arange(len(songs)) // 40):
    df['weight'] = ranking
    ranking -= 1
    chunks.append(df)
    
chunks = pd.concat(chunks)
chunks.to_csv("song_features")

/home/saman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
